In [1]:
import openeo
import numpy as np

url = "https://openeo-staging.dataspace.copernicus.eu"
connection = openeo.connect(url).authenticate_oidc()

Authenticated using refresh token.


In [15]:
datacube = connection.datacube_from_process("insar_preprocessing_v02", 
                                 burst_id = 249435, 
                                 sub_swath = "IW2",
                                 temporal_extent = ["2024-08-09", "2024-09-02"],
                                 master_date = "2024-09-02",
                                 polarization = "vv"
                                ).save_result("GTiff")


job = datacube.create_job(title="sample_preprocessing_vv")
job.start_job()

<BatchJob job_id='j-2509040751314022b5f9e9233d397d92'>

In [16]:
job = connection.job("j-2509040751314022b5f9e9233d397d92")
job

<BatchJob job_id='j-2509040751314022b5f9e9233d397d92'>

Re use the pre-processed Sentinel-1 SLC data via STAC:

In [2]:
cube = connection.load_stac_from_job("j-2509040751314022b5f9e9233d397d92")

In [3]:
cube.metadata.band_names

['i_VV', 'q_VV', 'grid_lat', 'grid_lon']

In [4]:
from openeo.processes import ProcessBuilder, array_element, array_create

def compute_parts_VV(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VV_mst")
    q_mst = array_element(x,label="q_VV_mst")
    i_slv = array_element(x,label="i_VV_slv")
    q_slv = array_element(x,label="q_VV_slv")
    return array_create([i_mst*i_slv + q_mst*q_slv,q_mst*i_slv - i_mst*q_slv,i_mst**2+q_mst**2,i_slv**2+q_slv**2])
    
def compute_parts_VH(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VH_mst")
    q_mst = array_element(x,label="q_VH_mst")
    i_slv = array_element(x,label="i_VH_slv")
    q_slv = array_element(x,label="q_VH_slv")
    return array_create([i_mst*i_slv + q_mst*q_slv,q_mst*i_slv - i_mst*q_slv,i_mst**2+q_mst**2,i_slv**2+q_slv**2])
    
def compute_coherence_amplitude_VV(x:ProcessBuilder):
    q_num_VV = array_element(x,label="q_num_VV")
    i_num_VV = array_element(x,label="i_num_VV")
    mst_den = array_element(x,label="mst_den")
    slv_den = array_element(x,label="slv_den")
    
    coh_i = i_num_VV / ((mst_den * slv_den + 1e-12)**0.5)
    coh_q = q_num_VV / ((mst_den * slv_den + 1e-12)**0.5)
    
    return (coh_i**2+coh_q**2)**0.5

def compute_coherence_amplitude_VH(x:ProcessBuilder):
    q_num_VH = array_element(x,label="q_num_VH")
    i_num_VH = array_element(x,label="i_num_VH")
    mst_den = array_element(x,label="mst_den")
    slv_den = array_element(x,label="slv_den")
    
    coh_i = i_num_VH / ((mst_den * slv_den + 1e-12)**0.5)
    coh_q = q_num_VH / ((mst_den * slv_den + 1e-12)**0.5)
    
    return (coh_i**2+coh_q**2)**0.5

In [5]:
dates = [np.datetime64("2024-08-09"),np.datetime64("2024-08-21"),np.datetime64("2024-09-02")]
dates_pair = [[dates[0],dates[1]],[dates[1],dates[2]]]
# We use a loop over the dates we want to request and a combination of filter_temporal + merge_cubes to get only the dates we want:
for i,date_pair in enumerate(dates_pair):
    # The requested date until the day after, since openEO excludes the end date
    mst_date = date_pair[0]
    slv_date = date_pair[1]
    mst_temporal_extent = [str(mst_date),str(mst_date + np.timedelta64(1, "D"))]
    slv_temporal_extent = [str(slv_date),str(slv_date + np.timedelta64(1, "D"))]

    if i == 0:
        S1_slant_range_mst_bands = cube.filter_temporal(mst_temporal_extent)
        S1_slant_range_slv_bands = cube.filter_temporal(slv_temporal_extent)
        break # +++++++++++++++++++++ Do it only for one pair for testing
    else:
        S1_slant_range_mst_bands = cube.filter_temporal(mst_temporal_extent).merge_cubes(S1_slant_range_mst_bands)
        S1_slant_range_slv_bands = cube.filter_temporal(slv_temporal_extent).merge_cubes(S1_slant_range_slv_bands)
        

In [6]:
S1_slant_range_mst_bands = S1_slant_range_mst_bands.filter_bands([0,1]).drop_dimension("t") # "i_" real part,"q_" imaginary part
S1_slant_range_slv_bands = S1_slant_range_slv_bands.filter_bands([0,1]).drop_dimension("t")

In [7]:
S1_slant_range_mst_bands = S1_slant_range_mst_bands.rename_labels(dimension="bands",
                                                                  source=[0,1],
                                                                  target=["i_VV_mst","q_VV_mst"])
S1_slant_range_slv_bands = S1_slant_range_slv_bands.rename_labels(dimension="bands",
                                                                  source=[0,1],
                                                                  target=["i_VV_slv","q_VV_slv"])

In [8]:
S1_slant_range_mst_slv_bands = S1_slant_range_mst_bands.merge_cubes(S1_slant_range_slv_bands)

In [9]:
coh_numerator_denominator_VV = S1_slant_range_mst_slv_bands.apply_dimension(dimension="bands",
                                                               process=compute_parts_VV).rename_labels("bands",target=["i_num_VV","q_num_VV","mst_den","slv_den"])

In [10]:
coh_numerator_denominator_VV.execute_batch()

0:00:00 Job 'j-2508200943394ff6a90f978edeb0d7ad': send 'start'
0:00:14 Job 'j-2508200943394ff6a90f978edeb0d7ad': created (progress 0%)
0:00:20 Job 'j-2508200943394ff6a90f978edeb0d7ad': created (progress 0%)
0:00:26 Job 'j-2508200943394ff6a90f978edeb0d7ad': created (progress 0%)
0:00:34 Job 'j-2508200943394ff6a90f978edeb0d7ad': running (progress 4.9%)
0:00:44 Job 'j-2508200943394ff6a90f978edeb0d7ad': running (progress 6.4%)
0:00:57 Job 'j-2508200943394ff6a90f978edeb0d7ad': running (progress 8.2%)
0:01:12 Job 'j-2508200943394ff6a90f978edeb0d7ad': running (progress 10.3%)
0:01:32 Job 'j-2508200943394ff6a90f978edeb0d7ad': running (progress 12.8%)
0:01:56 Job 'j-2508200943394ff6a90f978edeb0d7ad': finished (progress 100%)


<BatchJob job_id='j-2508200943394ff6a90f978edeb0d7ad'>

In [9]:
from openeo.processes import inspect

In [10]:
# Boxcar filter
range_dim   = 5
azimuth_dim = 1

# Create kernel
kernel = np.ones((range_dim,azimuth_dim))

# Pad with zeros to get a square kernel
pad = int((range_dim - azimuth_dim) / 2)
kernel = np.pad(kernel,((0,0),(pad,pad)))

coh_numerator_denominator_VV_boxcar = coh_numerator_denominator_VV.apply_kernel(kernel=kernel,
                                                                                factor=1/(range_dim*azimuth_dim),
                                                                                border=0)

coh_VV = coh_numerator_denominator_VV_boxcar.reduce_dimension(dimension="bands",
                                                              reducer=compute_coherence_amplitude_VV)

coordinate_bands = cube.filter_temporal(mst_temporal_extent).filter_bands([2,3])\
    .drop_dimension("t").rename_labels(dimension="bands", target=["latitude","longitude"])

coh_amplitude = coh_VV.add_dimension(name="bands",type="bands",label="coh_VV_amp")\
    .merge_cubes(coordinate_bands).add_dimension(name="t", type="temporal", label="2020-05-01T00:00:00Z")

coh_amplitude = coh_amplitude.resample_spatial(projection="EPSG:32632",resolution=20.0,method="geocode").save_result(format="GTiff")

job_options = {
    "image-name": "registry.stag.waw3-1.openeo-int.v1.dataspace.copernicus.eu/dev/openeo-geotrellis-kube-python311:20250903-133",
    "openeo-jar-path": "https://artifactory.vgt.vito.be/artifactory/libs-release-public/org/openeo/geotrellis-extensions/geocode/geotrellis-extensions-geocode.jar"
}
job = coh_amplitude.create_job(title="ClouDInSAR_coherence_from_preprocessing", job_options=job_options)

In [11]:
job.start_job()

<BatchJob job_id='j-2509040820244b149b936538719494b4'>

In [12]:
job

<BatchJob job_id='j-2509040820244b149b936538719494b4'>

In [24]:
#job = connection.job("j-25060509012543ebb48aa5ee8c65b7ed")
job.get_results().download_files()

No assets found in job result metadata.


[PosixPath('/home/driesj/code/notebooks/s1-workflows/notebooks/job-results.json')]